In [1]:
import numpy as np

## jupyter notebook showing how to inject events with genie
This is fairly slow, and can be cpu intensive, so don't simulate more than 1,000 events like this. For large datasets use slurm with the py script

#### also the following cell was just to resolve some annoying import problems I got with vs code's conda environment

In [2]:
import os
import ctypes
import sys


modules_to_clean = []
for name in list(sys.modules.keys()):
    if any(substring in name for substring in ['h5py', 'prometheus', '_hdf5', '_h5py', '_errors']):
        modules_to_clean.append(name)

for mod in modules_to_clean:
    if mod in sys.modules:
        del sys.modules[mod]

# library path
intel_paths = [
    "/software/local/intel/parallel_studio_xe_2020_upd4/compilers_and_libraries_2020.4.304/linux/compiler/lib/intel64",
    "/software/local/intel/parallel_studio_xe_2020_upd4/compilers_and_libraries_2020.4.304/linux/mkl/lib/intel64",
]

current_ld_path = os.environ.get('LD_LIBRARY_PATH', '')
new_ld_path = ':'.join(intel_paths + [current_ld_path])
os.environ['LD_LIBRARY_PATH'] = new_ld_path

intel_libs = [
    "/software/local/intel/parallel_studio_xe_2020_upd4/compilers_and_libraries_2020.4.304/linux/compiler/lib/intel64/libimf.so",
    "/software/local/intel/parallel_studio_xe_2020_upd4/compilers_and_libraries_2020.4.304/linux/compiler/lib/intel64/libintlc.so.5",
    "/software/local/intel/parallel_studio_xe_2020_upd4/compilers_and_libraries_2020.4.304/linux/compiler/lib/intel64/libsvml.so",
]

for lib_path in intel_libs:
    try:
        ctypes.CDLL(lib_path, mode=ctypes.RTLD_GLOBAL)
        print(f"Successfully preloaded: {lib_path}")
    except Exception as e:
        print(f"Failed to preload {lib_path}: {e}")

# # Python paths
sys.path.insert(0, '/groups/icecube/jackp/prometheus_genie_cleaned/harvard-prometheus')
sys.path.insert(0, '/groups/icecube/jackp/prometheus_genie_cleaned/harvard-prometheus/genie_examples')

try:
    import h5py
    print("✓ h5py imported successfully")
except Exception as e:
    print(f"✗ h5py import failed: {e}")

try:
    from prometheus import Prometheus, config
    print("✓ prometheus imported successfully!")
except Exception as e:
    print(f"✗ prometheus import failed: {e}")
    import traceback
    traceback.print_exc()

Successfully preloaded: /software/local/intel/parallel_studio_xe_2020_upd4/compilers_and_libraries_2020.4.304/linux/compiler/lib/intel64/libimf.so
Successfully preloaded: /software/local/intel/parallel_studio_xe_2020_upd4/compilers_and_libraries_2020.4.304/linux/compiler/lib/intel64/libintlc.so.5
Successfully preloaded: /software/local/intel/parallel_studio_xe_2020_upd4/compilers_and_libraries_2020.4.304/linux/compiler/lib/intel64/libsvml.so
✓ h5py imported successfully
✓ prometheus imported successfully!


In [3]:
from prometheus import Prometheus, config

In [4]:



import pandas as pd
import numpy as np
import time
import sys
import argparse
import os
import logging
from datetime import datetime

sys.path.append('../')
from prometheus import Prometheus, config
import prometheus
import gc

from genie_parser_injection import parse_and_convert_genie
from aggregate_hadronic_shower import group_hadronic_showers
from inject_in_cylinder import inject_particles_in_cylinder
from rotate_particles import rotate_particles_final
from gevgen_injection import run_gevgen, convert_to_gst, convert_to_gtrac

logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)
RESOURCE_DIR = f"{'/'.join(prometheus.__path__[0].split('/')[:-1])}/resources/"
OUTPUT_DIR = f"{'/'.join(prometheus.__path__[0].split('/')[:-1])}/genie_examples/output"


In [5]:
num_events = 5 ## just a short dataset for testing
simset = 2
start_time = time.time()


start_time = time.time()


print(f"Using simset: {simset}")
processing_start_time = time.time()



logger.info("Starting GENIE simulation")
genie_start = time.time()
ghep_file = run_gevgen(num_events, OUTPUT_DIR)### TODO: add extra parameters to this in addition to just num_events

gtrac_file = convert_to_gtrac(ghep_file, OUTPUT_DIR)

genie_time = time.time() - genie_start
logger.info(f"GENIE simulation completed in {genie_time:.2f} seconds")
print('about to')
prometheus_set, primary_set = parse_and_convert_genie(gtrac_file)
print('supposedly finished')
prometheus_set_pre_injection, primary_set_pre_injection = prometheus_set.copy(), primary_set.copy()
processing_end_time = time.time()
print(f"File processing and conversion completed in {processing_end_time - processing_start_time:.2f} seconds")

primrary_file_path = f'{OUTPUT_DIR}/genie_events_primary_new_test.parquet' ## could make these unique if you don't want them to be overwritten
prometheus_file_path = f'{OUTPUT_DIR}/genie_events_prometheus_new_test.parquet'

# inject into cylinder:
primary_set, prometheus_set = inject_particles_in_cylinder(primary_set, ## neutrino information
                    prometheus_set, ## child particle information
                    cylinder_radius=500, ## meters
                    cylinder_height = 1000, # meters
                    cylinder_center = (0, 0, 0), # meters not including offset
                    )

primary_set, prometheus_set = rotate_particles_final(primary_set, prometheus_set)
# prometheus_set  = group_hadronic_showers(prometheus_set) ## aggregate !


save_start_time = time.time()
# position arrays to serialized strings
prometheus_set['position'] = prometheus_set['position'].apply(lambda x: [arr.tolist() for arr in x])
num_events = len(prometheus_set)
print(f"Processing {num_events} events")
prometheus_set.to_parquet(prometheus_file_path)
primary_set.to_parquet(primrary_file_path)
save_end_time = time.time()
print(f"Serialization and saving completed in {save_end_time - save_start_time:.2f} seconds")

## GENIE stuff:
config["injection"]["name"] = "GENIE"
config["run"]["outfile"] = f"{OUTPUT_DIR}/{num_events}_events_simset_{simset}.parquet"
config["run"]["nevents"] = num_events
config["injection"]["GENIE"]["inject"] = False ## ideally this should toggle using gevgen and be true, but for now setting this to false helps solve a headache in promtheus.py
config["injection"]["GENIE"]["simulation"] = {}
## geofile:
config["detector"]["geo file"] = f"{RESOURCE_DIR}/geofiles/icecube.geo"
## ppc configuration - using the 'new' ppc:
config['photon propagator']['name'] = 'PPC_UPGRADE'
config["photon propagator"]["PPC_UPGRADE"]["paths"]["ppc_tmpdir"] = "./ppc_tmpdir"+str(simset)
config["photon propagator"]["PPC_UPGRADE"]["paths"]["ppc_tmpfile"] = "ppc_tmp"+str(simset)
print(config["photon propagator"])
config["photon propagator"]["PPC_UPGRADE"]["simulation"]["supress_output"] = True ## just for printing, can turn to True if you don't want a cluttered print out!

# sim_start_time = time.time()
# p = Prometheus(config, primary_set_parquet_path=primrary_file_path, prometheus_set_parquet_path=prometheus_file_path)
# p.sim()
# sim_end_time = time.time()
# print(f"Prometheus simulation completed in {sim_end_time - sim_start_time:.2f} seconds")

# # end timing
# end_time = time.time()
# total_time = end_time - start_time
# print(f"Total execution time: {total_time:.2f} seconds ({total_time/60:.2f} minutes)")
# print('Finished without catastrophic error')



2025-06-19 13:36:40,354 - __main__ - INFO - Starting GENIE simulation
2025-06-19 13:36:40,355 - gevgen_injection - INFO - Running GEVGEN with 5 events
2025-06-19 13:36:40,356 - gevgen_injection - INFO - genie command: ['/groups/icecube/jackp/genie-3.4.2/bin/gevgen', '-n', '5', '-p', '14', '-t', '1000080160[0.888],1000010010[0.112]', '-e', '5,40', '-f', 'x^-2', '--seed', '16', '--cross-sections', '/groups/icecube/jackp/genie-3.4.2/ice_numu_cross_sections.xml', '--event-record-print-level', '3', '-o', '/groups/icecube/jackp/prometheus_genie_cleaned/harvard-prometheus/genie_examples/output/gntp_icecube_5_events.ghep.root', '--debug-flags', 'PhysModels,EventGen,NucleonDecay', '--tune', 'G18_02a_00_000']


Using simset: 2
cmd:  ['/groups/icecube/jackp/genie-3.4.2/bin/gevgen', '-n', '5', '-p', '14', '-t', '1000080160[0.888],1000010010[0.112]', '-e', '5,40', '-f', 'x^-2', '--seed', '16', '--cross-sections', '/groups/icecube/jackp/genie-3.4.2/ice_numu_cross_sections.xml', '--event-record-print-level', '3', '-o', '/groups/icecube/jackp/prometheus_genie_cleaned/harvard-prometheus/genie_examples/output/gntp_icecube_5_events.ghep.root', '--debug-flags', 'PhysModels,EventGen,NucleonDecay', '--tune', 'G18_02a_00_000']


2025-06-19 13:36:42,975 - gevgen_injection - INFO - GEVGEN completed. Output: /groups/icecube/jackp/prometheus_genie_cleaned/harvard-prometheus/genie_examples/output/gntp_icecube_5_events.ghep.root
2025-06-19 13:36:42,976 - gevgen_injection - INFO - Converting to GTRAC format using rootracker




*****************************************************************************************************************
*                                                                                                               *
*                                                   .oooooo.    oooooooooooo ooooo      ooo ooooo oooooooooooo  *
*                                                  d8P'  `Y8b   `888'     `8 `888b.     `8' `888' `888'     `8  *
*                                                 888            888          8 `88b.    8   888   888          * 
*                        Ndyooym          dN      888            888oooo8     8   `88b.  8   888   888oooo8     *
*                     Nds//+sdmoy       d+m       888     ooooo  888    "     8     `88b.8   888   888    "     * 
*                   Nh+//ohN  m+s      N//syyN    `88.    .88'   888       o  8       `888   888   888       o  *
*                 Ny+//od   Nh+oN       o///+      `Y8bood8P'   o888ooooood8 o8o    

2025-06-19 13:36:44,337 - gevgen_injection - INFO - GTRAC conversion completed. Output: /groups/icecube/jackp/prometheus_genie_cleaned/harvard-prometheus/genie_examples/output/ice_cube_sim.gtrac.root
2025-06-19 13:36:44,338 - __main__ - INFO - GENIE simulation completed in 3.98 seconds
2025-06-19 13:36:44,340 - fsspec.local - DEBUG - open file: /groups/icecube/jackp/prometheus_genie_cleaned/harvard-prometheus/genie_examples/output/ice_cube_sim.gtrac.root
2025-06-19 13:36:44,341 - fsspec.local - DEBUG - open file: /groups/icecube/jackp/prometheus_genie_cleaned/harvard-prometheus/genie_examples/output/ice_cube_sim.gtrac.root
2025-06-19 13:36:44,342 - fsspec.local - DEBUG - open file: /groups/icecube/jackp/prometheus_genie_cleaned/harvard-prometheus/genie_examples/output/ice_cube_sim.gtrac.root
2025-06-19 13:36:44,343 - fsspec.local - DEBUG - open file: /groups/icecube/jackp/prometheus_genie_cleaned/harvard-prometheus/genie_examples/output/ice_cube_sim.gtrac.root
2025-06-19 13:36:44,372 -

about to
=== ALL TREES/OBJECTS IN ROOT FILE ===
  gRooTracker;1: <class 'uproot.models.TTree.Model_TTree_v20'>
    Branches: ['EvtFlags', 'EvtFlags/TObject', 'EvtFlags/fNbits', 'EvtFlags/fNbytes', 'EvtFlags/fAllBits', 'EvtCode', 'EvtCode/TObject', 'EvtCode/fString', 'EvtNum', 'EvtXSec', 'EvtDXSec', 'EvtKPS', 'EvtWght', 'EvtProb', 'EvtVtx', 'StdHepN', 'StdHepPdg', 'StdHepStatus', 'StdHepRescat', 'StdHepX4', 'StdHepP4', 'StdHepPolz', 'StdHepFd', 'StdHepLd', 'StdHepFm', 'StdHepLm']
=== DETAILED ROOT FILE EXPLORATION ===

Branches in gRooTracker tree:

Branch: EvtFlags
  Could not read data: none of the rules matched
in file /groups/icecube/jackp/prometheus_genie_cleaned/harvard-prometheus/genie_examples/output/ice_cube_sim.gtrac.root
in object /gRooTracker;1:EvtFlags/TObject

Branch: EvtFlags/TObject
  Could not read data: none of the rules matched
in file /groups/icecube/jackp/prometheus_genie_cleaned/harvard-prometheus/genie_examples/output/ice_cube_sim.gtrac.root
in object /gRooTracker

/groups/icecube/jackp/prometheus_genie_cleaned/harvard-prometheus/genie_examples/genie_parser_injection.py:23: RuntimeWarning: invalid value encountered in scalar divide
  angle = np.arccos(np.clip(np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2)), -1, 1))


#### the gevgen injection happens in file, gevgen_injection.py and is called by helper functions in cell below 

In [5]:

def main(num_events, simset=1):
    # timing
    start_time = time.time()


    print(f"Using simset: {simset}")
    processing_start_time = time.time()



    logger.info("Starting GENIE simulation")
    genie_start = time.time()
    ghep_file = run_gevgen(num_events, OUTPUT_DIR)### TODO: add extra parameters to this in addition to just num_events
    gtrac_file = convert_to_gtrac(ghep_file, OUTPUT_DIR)
    
    genie_time = time.time() - genie_start
    logger.info(f"GENIE simulation completed in {genie_time:.2f} seconds")
    prometheus_set, primary_set = parse_and_convert_genie(gtrac_file)
    prometheus_set_pre_injection, primary_set_pre_injection = prometheus_set.copy(), primary_set.copy()
    processing_end_time = time.time()
    print(f"File processing and conversion completed in {processing_end_time - processing_start_time:.2f} seconds")

    primrary_file_path = f'{OUTPUT_DIR}/genie_events_primary_new_test.parquet' ## could make these unique if you don't want them to be overwritten
    prometheus_file_path = f'{OUTPUT_DIR}/genie_events_prometheus_new_test.parquet'

    # inject into cylinder:
    primary_set, prometheus_set = inject_particles_in_cylinder(primary_set, ## neutrino information
                        prometheus_set, ## child particle information
                        cylinder_radius=500, ## meters
                        cylinder_height = 1000, # meters
                        cylinder_center = (0, 0, 0), # meters not including offset
                        )

    primary_set, prometheus_set = rotate_particles_final(primary_set, prometheus_set)
    prometheus_set  = group_hadronic_showers(prometheus_set) ## aggregate !


    save_start_time = time.time()
    # position arrays to serialized strings
    prometheus_set['position'] = prometheus_set['position'].apply(lambda x: [arr.tolist() for arr in x])
    num_events = len(prometheus_set)
    print(f"Processing {num_events} events")
    prometheus_set.to_parquet(prometheus_file_path)
    primary_set.to_parquet(primrary_file_path)
    save_end_time = time.time()
    print(f"Serialization and saving completed in {save_end_time - save_start_time:.2f} seconds")

    ## GENIE stuff:
    config["injection"]["name"] = "GENIE"
    config["run"]["outfile"] = f"{OUTPUT_DIR}/{num_events}_events_simset_{simset}.parquet"
    config["run"]["nevents"] = num_events
    config["injection"]["GENIE"]["inject"] = False ## ideally this should toggle using gevgen and be true, but for now setting this to false helps solve a headache in promtheus.py
    config["injection"]["GENIE"]["simulation"] = {}
    ## geofile:
    config["detector"]["geo file"] = f"{RESOURCE_DIR}/geofiles/icecube.geo"
    ## ppc configuration - using the 'new' ppc:
    config['photon propagator']['name'] = 'PPC_UPGRADE'
    config["photon propagator"]["PPC_UPGRADE"]["paths"]["ppc_tmpdir"] = "./ppc_tmpdir"+str(simset)
    config["photon propagator"]["PPC_UPGRADE"]["paths"]["ppc_tmpfile"] = "ppc_tmp"+str(simset)
    print(config["photon propagator"])
    config["photon propagator"]["PPC_UPGRADE"]["simulation"]["supress_output"] = True ## just for printing, can turn to True if you don't want a cluttered print out!

    sim_start_time = time.time()
    p = Prometheus(config, primary_set_parquet_path=primrary_file_path, prometheus_set_parquet_path=prometheus_file_path)
    p.sim()
    sim_end_time = time.time()
    print(f"Prometheus simulation completed in {sim_end_time - sim_start_time:.2f} seconds")
    
    # end timing
    end_time = time.time()
    total_time = end_time - start_time
    print(f"Total execution time: {total_time:.2f} seconds ({total_time/60:.2f} minutes)")
    print('Finished without catastrophic error')
    return




In [6]:
# num_events = 10 ## just a short dataset for testing
# simset = 219
# print("--------------------------------------------------------------")
# print("--------------------------------------------------------------")
# print("Launching simulation")
# main(num_events, simset)
# print("Finished call")
# print("--------------------------------------------------------------")
# print("--------------------------------------------------------------")

In [7]:
import pickle

In [8]:

num_events = 5 ## just a short dataset for testing
simset = 2
start_time = time.time()


print(f"Using simset: {simset}")
processing_start_time = time.time()



logger.info("Starting GENIE simulation")
genie_start = time.time()
ghep_file = run_gevgen(num_events, OUTPUT_DIR)### TODO: add extra parameters to this in addition to just num_events
gtrac_file = convert_to_gtrac(ghep_file, OUTPUT_DIR)

genie_time = time.time() - genie_start
logger.info(f"GENIE simulation completed in {genie_time:.2f} seconds")
prometheus_set, primary_set = parse_and_convert_genie(gtrac_file)

processing_end_time = time.time()
print(f"File processing and conversion completed in {processing_end_time - processing_start_time:.2f} seconds")

primrary_file_path = f'{OUTPUT_DIR}/genie_events_primary_new_test.parquet' ## could make these unique if you don't want them to be overwritten
prometheus_file_path = f'{OUTPUT_DIR}/genie_events_prometheus_new_test.parquet'
#prometheus_set_pre_injection, primary_set_pre_injection = prometheus_set.copy(), primary_set.copy()
# inject into cylinder:

print(prometheus_set.iloc[0])  # Should show zeros

# Copy immediately before injection
prometheus_set_pre_injection = prometheus_set.copy(deep=True)
primary_set_pre_injection = primary_set.copy(deep=True)
import pickle

# Save the original data using pickle (handles complex nested arrays perfectly)
with open('prometheus_pre_injection.pkl', 'wb') as f:
    pickle.dump(prometheus_set, f)
    
with open('primary_pre_injection.pkl', 'wb') as f:
    pickle.dump(primary_set, f)
primary_set, prometheus_set = inject_particles_in_cylinder(primary_set, ## neutrino information
                    prometheus_set, ## child particle information
                    cylinder_radius=500, ## meters
                    cylinder_height = 1000, # meters
                    cylinder_center = (0, 0, 0), # meters not including offset
                    )

primary_set, prometheus_set = rotate_particles_final(primary_set, prometheus_set)
#prometheus_set  = group_hadronic_showers(prometheus_set) ## aggregate !

print("After injection:")
print(f"Original pos_x: {prometheus_set['pos_x'].iloc[0]}")
print(f"Copy pos_x: {prometheus_set_pre_injection['pos_x'].iloc[0]}")

save_start_time = time.time()
# position arrays to serialized strings
prometheus_set['position'] = prometheus_set['position'].apply(lambda x: [arr.tolist() for arr in x])
num_events = len(prometheus_set)
print(f"Processing {num_events} events")
prometheus_set.to_parquet(prometheus_file_path)
primary_set.to_parquet(primrary_file_path)
save_end_time = time.time()
print(f"Serialization and saving completed in {save_end_time - save_start_time:.2f} seconds")

## GENIE stuff:
config["injection"]["name"] = "GENIE"
config["run"]["outfile"] = f"{OUTPUT_DIR}/{num_events}_events_simset_{simset}.parquet"
config["run"]["nevents"] = num_events
config["injection"]["GENIE"]["inject"] = False ## ideally this should toggle using gevgen and be true, but for now setting this to false helps solve a headache in promtheus.py
config["injection"]["GENIE"]["simulation"] = {}
## geofile:
config["detector"]["geo file"] = f"{RESOURCE_DIR}/geofiles/icecube.geo"
## ppc configuration - using the 'new' ppc:
config['photon propagator']['name'] = 'PPC_UPGRADE'
config["photon propagator"]["PPC_UPGRADE"]["paths"]["ppc_tmpdir"] = "./ppc_tmpdir"+str(simset)
config["photon propagator"]["PPC_UPGRADE"]["paths"]["ppc_tmpfile"] = "ppc_tmp"+str(simset)
print(config["photon propagator"])
config["photon propagator"]["PPC_UPGRADE"]["simulation"]["supress_output"] = True ## just for printing, can turn to True if you don't want a cluttered print out!

sim_start_time = time.time()
p = Prometheus(config, primary_set_parquet_path=primrary_file_path, prometheus_set_parquet_path=prometheus_file_path)
p.sim()
sim_end_time = time.time()
print(f"Prometheus simulation completed in {sim_end_time - sim_start_time:.2f} seconds")

# end timing
end_time = time.time()
total_time = end_time - start_time
print(f"Total execution time: {total_time:.2f} seconds ({total_time/60:.2f} minutes)")
print('Finished without catastrophic error')





2025-06-05 14:47:33,374 - __main__ - INFO - Starting GENIE simulation
2025-06-05 14:47:33,375 - gevgen_injection - INFO - Running GEVGEN with 5 events
2025-06-05 14:47:33,375 - gevgen_injection - INFO - genie command: ['/groups/icecube/jackp/genie-3.4.2/bin/gevgen', '-n', '5', '-p', '14', '-t', '1000080160[0.888],1000010010[0.112]', '-e', '5,40', '-f', 'x^-2', '--seed', '16', '--cross-sections', '/groups/icecube/jackp/genie-3.4.2/ice_numu_cross_sections.xml', '--event-record-print-level', '3', '-o', '/groups/icecube/jackp/prometheus_genie_cleaned/harvard-prometheus/genie_examples/output/gntp_icecube_5_events.ghep.root', '--debug-flags', 'PhysModels,EventGen,NucleonDecay', '--tune', 'G18_02a_00_000']


Using simset: 2
cmd:  ['/groups/icecube/jackp/genie-3.4.2/bin/gevgen', '-n', '5', '-p', '14', '-t', '1000080160[0.888],1000010010[0.112]', '-e', '5,40', '-f', 'x^-2', '--seed', '16', '--cross-sections', '/groups/icecube/jackp/genie-3.4.2/ice_numu_cross_sections.xml', '--event-record-print-level', '3', '-o', '/groups/icecube/jackp/prometheus_genie_cleaned/harvard-prometheus/genie_examples/output/gntp_icecube_5_events.ghep.root', '--debug-flags', 'PhysModels,EventGen,NucleonDecay', '--tune', 'G18_02a_00_000']


2025-06-05 14:47:35,751 - gevgen_injection - INFO - GEVGEN completed. Output: /groups/icecube/jackp/prometheus_genie_cleaned/harvard-prometheus/genie_examples/output/gntp_icecube_5_events.ghep.root
2025-06-05 14:47:35,752 - gevgen_injection - INFO - Converting to GTRAC format using rootracker




*****************************************************************************************************************
*                                                                                                               *
*                                                   .oooooo.    oooooooooooo ooooo      ooo ooooo oooooooooooo  *
*                                                  d8P'  `Y8b   `888'     `8 `888b.     `8' `888' `888'     `8  *
*                                                 888            888          8 `88b.    8   888   888          * 
*                        Ndyooym          dN      888            888oooo8     8   `88b.  8   888   888oooo8     *
*                     Nds//+sdmoy       d+m       888     ooooo  888    "     8     `88b.8   888   888    "     * 
*                   Nh+//ohN  m+s      N//syyN    `88.    .88'   888       o  8       `888   888   888       o  *
*                 Ny+//od   Nh+oN       o///+      `Y8bood8P'   o888ooooood8 o8o    

2025-06-05 14:47:37,070 - gevgen_injection - INFO - GTRAC conversion completed. Output: /groups/icecube/jackp/prometheus_genie_cleaned/harvard-prometheus/genie_examples/output/ice_cube_sim.gtrac.root
2025-06-05 14:47:37,072 - __main__ - INFO - GENIE simulation completed in 3.70 seconds
2025-06-05 14:47:37,074 - fsspec.local - DEBUG - open file: /groups/icecube/jackp/prometheus_genie_cleaned/harvard-prometheus/genie_examples/output/ice_cube_sim.gtrac.root
2025-06-05 14:47:37,075 - fsspec.local - DEBUG - open file: /groups/icecube/jackp/prometheus_genie_cleaned/harvard-prometheus/genie_examples/output/ice_cube_sim.gtrac.root
2025-06-05 14:47:37,076 - fsspec.local - DEBUG - open file: /groups/icecube/jackp/prometheus_genie_cleaned/harvard-prometheus/genie_examples/output/ice_cube_sim.gtrac.root
2025-06-05 14:47:37,077 - fsspec.local - DEBUG - open file: /groups/icecube/jackp/prometheus_genie_cleaned/harvard-prometheus/genie_examples/output/ice_cube_sim.gtrac.root
2025-06-05 14:47:37,107 -

=== RAW GENIE EVENT ANALYSIS ===
Event 0:
  Particle 0: PDG=14, Status='initial'
  Particle 1: PDG=2212, Status='initial'
  Particle 2: PDG=13, Status='final'
  Particle 3: PDG=2000000001, Status='DIS pre-frag hadronic state'
  Particle 4: PDG=2, Status='DIS pre-frag hadronic state'
  Particle 5: PDG=2203, Status='DIS pre-frag hadronic state'
  Particle 6: PDG=92, Status='DIS pre-frag hadronic state'
  Particle 7: PDG=211, Status='final'
  Particle 8: PDG=-211, Status='final'
  Particle 9: PDG=2212, Status='final'
  Particle 10: PDG=213, Status='DIS pre-frag hadronic state'
  Particle 11: PDG=211, Status='final'
  Particle 12: PDG=111, Status='final'
Event 1:
  Particle 0: PDG=14, Status='initial'
  Particle 1: PDG=1000080160, Status='initial'
  Particle 2: PDG=2112, Status='nucleon target'
  Particle 3: PDG=1000080150, Status='intermediate'
  Particle 4: PDG=13, Status='final'
  Particle 5: PDG=2000000001, Status='DIS pre-frag hadronic state'
  Particle 6: PDG=2, Status='DIS pre-frag 

PpcTmpdirExistsError: ./ppc_tmpdir2 exists. Please remove it or specify force in the config

In [13]:
primary_set.iloc[0]

primary                                                      True
e                                                       32.736354
pdg_code                                                       14
interaction                                                    CC
theta                                                    0.564861
phi                                                      0.486335
bjorken_x                                                      -1
bjorken_y                                                      -1
pos_x                                                   182.73659
pos_y                                                 -441.175686
pos_z                                                 -186.409135
position        (182.73658952573766, -441.1756859232849, -186....
column_depth                                                   -1
custom_info     nu:14;tgt:1000010010;N:2212;q:1(v);proc:Weak[C...
Name: 0, dtype: object

In [14]:
primary_set.iloc[0]['custom_info']

'nu:14;tgt:1000010010;N:2212;q:1(v);proc:Weak[CC],DIS;'

In [15]:
primary_set.iloc[0]

primary                                                      True
e                                                       32.736354
pdg_code                                                       14
interaction                                                    CC
theta                                                    0.564861
phi                                                      0.486335
bjorken_x                                                      -1
bjorken_y                                                      -1
pos_x                                                   182.73659
pos_y                                                 -441.175686
pos_z                                                 -186.409135
position        (182.73658952573766, -441.1756859232849, -186....
column_depth                                                   -1
custom_info     nu:14;tgt:1000010010;N:2212;q:1(v);proc:Weak[C...
Name: 0, dtype: object

In [16]:
prometheus_set.iloc[0]

primary                [False, False, False, False, False, False]
e               [17.58709483881163, 11.361265253903209, 0.6156...
pdg_code                          [13, 211, -211, 2212, 211, 111]
interaction                              [CC, CC, CC, CC, CC, CC]
theta           [0.6915032423537506, 0.468718534521462, 0.5503...
phi             [0.4325069083495358, 0.6965414899497797, 1.698...
bjorken_x                    [-1.0, -1.0, -1.0, -1.0, -1.0, -1.0]
bjorken_y                    [-1.0, -1.0, -1.0, -1.0, -1.0, -1.0]
pos_x           [182.73658952573766, 182.73658952573766, 182.7...
pos_y           [-441.1756859232849, -441.1756859232849, -441....
pos_z           [-186.4091349946898, -186.4091349946898, -186....
position        [[182.73658952573766, -441.1756859232849, -186...
column_depth                 [-1.0, -1.0, -1.0, -1.0, -1.0, -1.0]
custom_info            [child, child, child, child, child, child]
Name: 0, dtype: object

In [18]:
# Load your saved data (assuming you used pickle method)
import pickle
import numpy as np

# Load pre-rotation data
with open('prometheus_pre_injection.pkl', 'rb') as f:
    prometheus_set_pre_injection = pickle.load(f)
    
with open('primary_pre_injection.pkl', 'rb') as f:
    primary_set_pre_injection = pickle.load(f)

# Current data is already rotated (prometheus_set, primary_set)

# Choose event to analyze
event_idx = 0

print(f"=== Analyzing Event {event_idx} ===\n")

# Extract pre-rotation data
theta_pre = prometheus_set_pre_injection.iloc[event_idx]['theta']
phi_pre = prometheus_set_pre_injection.iloc[event_idx]['phi']
nu_theta_pre = primary_set_pre_injection.iloc[event_idx]['theta']
nu_phi_pre = primary_set_pre_injection.iloc[event_idx]['phi']

# Extract post-rotation data  
theta_post = prometheus_set.iloc[event_idx]['theta']
phi_post = prometheus_set.iloc[event_idx]['phi']
nu_theta_post = primary_set.iloc[event_idx]['theta']
nu_phi_post = primary_set.iloc[event_idx]['phi']

print(f"Number of child particles: {len(theta_pre)}")
print(f"Primary neutrino θ: {nu_theta_pre:.4f} → {nu_theta_post:.4f}")
print(f"Primary neutrino φ: {nu_phi_pre:.4f} → {nu_phi_post:.4f}")
print()

# Convert spherical to cartesian for easier vector operations
def spherical_to_cartesian(theta, phi):
    """Convert spherical coordinates to unit vectors"""
    # Handle special case: θ=0 means pointing along +z axis, regardless of φ
    if abs(theta) < 1e-10:  # θ ≈ 0
        return np.array([0.0, 0.0, 1.0])
    elif abs(theta - np.pi) < 1e-10:  # θ ≈ π 
        return np.array([0.0, 0.0, -1.0])
    else:
        return np.array([
            np.sin(theta) * np.cos(phi),
            np.sin(theta) * np.sin(phi), 
            np.cos(theta)
        ])

# Primary neutrino vectors
# For pre-rotation: θ=0 means it's pointing along (0,0,1) regardless of φ value
nu_vec_pre = spherical_to_cartesian(nu_theta_pre, nu_phi_pre)
nu_vec_post = spherical_to_cartesian(nu_theta_post, nu_phi_post)

print("Primary neutrino vectors:")
print(f"Pre:  [{nu_vec_pre[0]:6.3f}, {nu_vec_pre[1]:6.3f}, {nu_vec_pre[2]:6.3f}]")
print(f"Post: [{nu_vec_post[0]:6.3f}, {nu_vec_post[1]:6.3f}, {nu_vec_post[2]:6.3f}]")
print()

# Child particle vectors
child_vecs_pre = []
child_vecs_post = []

for i in range(len(theta_pre)):
    if not np.isnan(theta_pre[i]):  # Only check theta, phi can be nan when theta=0
        child_vecs_pre.append(spherical_to_cartesian(theta_pre[i], phi_pre[i]))
        child_vecs_post.append(spherical_to_cartesian(theta_post[i], phi_post[i]))

child_vecs_pre = np.array(child_vecs_pre)
child_vecs_post = np.array(child_vecs_post)

print(f"Valid child particles: {len(child_vecs_pre)}")
print()

# Check 1: Relative angles between child particles should be preserved
print("=== Checking Inter-particle Angles ===")
for i in range(min(3, len(child_vecs_pre))):  # Check first 3 pairs
    for j in range(i+1, min(3, len(child_vecs_pre))):
        # Dot product gives cos(angle)
        cos_angle_pre = np.dot(child_vecs_pre[i], child_vecs_pre[j])
        cos_angle_post = np.dot(child_vecs_post[i], child_vecs_post[j])
        
        angle_pre = np.arccos(np.clip(cos_angle_pre, -1, 1)) * 180/np.pi
        angle_post = np.arccos(np.clip(cos_angle_post, -1, 1)) * 180/np.pi
        
        print(f"Particles {i}-{j}: {angle_pre:6.2f}° → {angle_post:6.2f}° (Δ = {abs(angle_pre-angle_post):.4f}°)")

print()

# Check 2: Angles relative to primary neutrino should be preserved
print("=== Checking Angles Relative to Primary Neutrino ===")
for i in range(min(3, len(child_vecs_pre))):
    cos_angle_pre = np.dot(child_vecs_pre[i], nu_vec_pre)
    cos_angle_post = np.dot(child_vecs_post[i], nu_vec_post)
    
    angle_pre = np.arccos(np.clip(cos_angle_pre, -1, 1)) * 180/np.pi
    angle_post = np.arccos(np.clip(cos_angle_post, -1, 1)) * 180/np.pi
    
    print(f"Child {i} to ν: {angle_pre:6.2f}° → {angle_post:6.2f}° (Δ = {abs(angle_pre-angle_post):.4f}°)")

print()

# Check 3: Cross products should have same magnitude (different direction is expected)
print("=== Checking Cross Product Magnitudes ===")
for i in range(min(2, len(child_vecs_pre))):
    for j in range(i+1, min(2, len(child_vecs_pre))):
        cross_pre = np.cross(child_vecs_pre[i], child_vecs_pre[j])
        cross_post = np.cross(child_vecs_post[i], child_vecs_post[j])
        
        mag_pre = np.linalg.norm(cross_pre)
        mag_post = np.linalg.norm(cross_post)
        
        print(f"Particles {i}×{j}: |cross| = {mag_pre:.4f} → {mag_post:.4f} (Δ = {abs(mag_pre-mag_post):.6f})")

print()

# Summary
print("=== Summary ===")
print("If rotation worked correctly:")
print("- Inter-particle angles should be identical (Δ ≈ 0)")
print("- Neutrino-relative angles should be identical (Δ ≈ 0)")  
print("- Cross product magnitudes should be identical (Δ ≈ 0)")
print("- Only the absolute directions should change, not relative geometry!")

# Bonus: Show the rotation effect
print(f"\n=== Rotation Effect ===")
print(f"Neutrino rotated from (0,0,1) to ({nu_vec_post[0]:.3f}, {nu_vec_post[1]:.3f}, {nu_vec_post[2]:.3f})")
rotation_angle = np.arccos(np.clip(np.dot([0,0,1], nu_vec_post), -1, 1)) * 180/np.pi
print(f"Total rotation angle: {rotation_angle:.2f}°")

=== Analyzing Event 0 ===

Number of child particles: 6
Primary neutrino θ: 0.0000 → 0.5649
Primary neutrino φ: nan → 0.4863

Primary neutrino vectors:
Pre:  [ 0.000,  0.000,  1.000]
Post: [ 0.473,  0.250,  0.845]

Valid child particles: 6

=== Checking Inter-particle Angles ===
Particles 0-1:  15.14° →  15.14° (Δ = 0.0000°)
Particles 0-2:  40.82° →  40.82° (Δ = 0.0000°)
Particles 1-2:  27.41° →  27.41° (Δ = 0.0000°)

=== Checking Angles Relative to Primary Neutrino ===
Child 0 to ν:   7.48° →   7.48° (Δ = 0.0000°)
Child 1 to ν:   8.09° →   8.09° (Δ = 0.0000°)
Child 2 to ν:  35.08° →  35.08° (Δ = 0.0000°)

=== Checking Cross Product Magnitudes ===
Particles 0×1: |cross| = 0.2611 → 0.2611 (Δ = 0.000000)

=== Summary ===
If rotation worked correctly:
- Inter-particle angles should be identical (Δ ≈ 0)
- Neutrino-relative angles should be identical (Δ ≈ 0)
- Cross product magnitudes should be identical (Δ ≈ 0)
- Only the absolute directions should change, not relative geometry!

=== Rotat

In [22]:
primary_set.iloc[0]

primary                                                      True
e                                                       32.736354
pdg_code                                                       14
interaction                                                    CC
theta                                                    0.564861
phi                                                      0.486335
bjorken_x                                                      -1
bjorken_y                                                      -1
pos_x                                                   182.73659
pos_y                                                 -441.175686
pos_z                                                 -186.409135
position        (182.73658952573766, -441.1756859232849, -186....
column_depth                                                   -1
custom_info     nu:14;tgt:1000010010;N:2212;q:1(v);proc:Weak[C...
Name: 0, dtype: object

In [23]:
prometheus_set.iloc[0]

primary                [False, False, False, False, False, False]
e               [17.58709483881163, 11.361265253903209, 0.6156...
pdg_code                          [13, 211, -211, 2212, 211, 111]
interaction                              [CC, CC, CC, CC, CC, CC]
theta           [0.6915032423537506, 0.468718534521462, 0.5503...
phi             [0.4325069083495358, 0.6965414899497797, 1.698...
bjorken_x                    [-1.0, -1.0, -1.0, -1.0, -1.0, -1.0]
bjorken_y                    [-1.0, -1.0, -1.0, -1.0, -1.0, -1.0]
pos_x           [182.73658952573766, 182.73658952573766, 182.7...
pos_y           [-441.1756859232849, -441.1756859232849, -441....
pos_z           [-186.4091349946898, -186.4091349946898, -186....
position        [[182.73658952573766, -441.1756859232849, -186...
column_depth                 [-1.0, -1.0, -1.0, -1.0, -1.0, -1.0]
custom_info            [child, child, child, child, child, child]
Name: 0, dtype: object

In [21]:
# Generate LaTeX tables for Event 7 pre and post rotation
import numpy as np
import pandas as pd

# Assuming you have the data loaded as before
event_idx = 0

# Extract data for event 7
theta_pre = prometheus_set_pre_injection.iloc[event_idx]['theta']
phi_pre = prometheus_set_pre_injection.iloc[event_idx]['phi'] 
energy_pre = prometheus_set_pre_injection.iloc[event_idx]['e']
pdg_pre = prometheus_set_pre_injection.iloc[event_idx]['pdg_code']
pos_pre = prometheus_set_pre_injection.iloc[event_idx]['position']

theta_post = prometheus_set.iloc[event_idx]['theta']
phi_post = prometheus_set.iloc[event_idx]['phi']
energy_post = prometheus_set.iloc[event_idx]['e']
pdg_post = prometheus_set.iloc[event_idx]['pdg_code']
pos_post = prometheus_set.iloc[event_idx]['position']

# Primary neutrino data
nu_theta_pre = primary_set_pre_injection.iloc[event_idx]['theta']
nu_phi_pre = primary_set_pre_injection.iloc[event_idx]['phi']
nu_energy_pre = primary_set_pre_injection.iloc[event_idx]['e']
nu_pos_pre = primary_set_pre_injection.iloc[event_idx]['position']

nu_theta_post = primary_set.iloc[event_idx]['theta']
nu_phi_post = primary_set.iloc[event_idx]['phi']
nu_energy_post = primary_set.iloc[event_idx]['e']
nu_pos_post = primary_set.iloc[event_idx]['position']

# Convert to cartesian for dot products
def spherical_to_cartesian(theta, phi):
    if abs(theta) < 1e-10:  # θ ≈ 0
        return np.array([0.0, 0.0, 1.0])
    else:
        return np.array([
            np.sin(theta) * np.cos(phi),
            np.sin(theta) * np.sin(phi), 
            np.cos(theta)
        ])

# Get direction vectors
nu_vec_pre = spherical_to_cartesian(nu_theta_pre, nu_phi_pre)
nu_vec_post = spherical_to_cartesian(nu_theta_post, nu_phi_post)

child_vecs_pre = []
child_vecs_post = []
for i in range(len(theta_pre)):
    if not np.isnan(theta_pre[i]):
        child_vecs_pre.append(spherical_to_cartesian(theta_pre[i], phi_pre[i]))
        child_vecs_post.append(spherical_to_cartesian(theta_post[i], phi_post[i]))

child_vecs_pre = np.array(child_vecs_pre)
child_vecs_post = np.array(child_vecs_post)

# Calculate dot products with neutrino
dots_pre = [np.dot(child_vecs_pre[i], nu_vec_pre) for i in range(len(child_vecs_pre))]
dots_post = [np.dot(child_vecs_post[i], nu_vec_post) for i in range(len(child_vecs_post))]

# Calculate cross products with neutrino (magnitude)
crosses_pre = [np.linalg.norm(np.cross(child_vecs_pre[i], nu_vec_pre)) for i in range(len(child_vecs_pre))]
crosses_post = [np.linalg.norm(np.cross(child_vecs_post[i], nu_vec_post)) for i in range(len(child_vecs_post))]

# PDG code names for readability
pdg_names = {13: r'$\mu^-$', 2212: r'$p$', 211: r'$\pi^+$', -211: r'$\pi^-$', 
             22: r'$\gamma$', 14: r'$\nu_\mu$', 2112: r'$n$'}

print("="*80)
print("PRE-ROTATION TABLE (Event 7)")
print("="*80)

pre_table = r"""
\begin{table}[h]
\centering
\caption{Event 7 - Pre-Rotation Particle Properties}
\begin{tabular}{|c|c|c|c|c|c|c|c|}
\hline
\textbf{Particle} & \textbf{PDG} & \textbf{Energy} & \textbf{$\theta$} & \textbf{$\phi$} & \textbf{$\vec{p} \cdot \hat{z}$} & \textbf{$|\vec{p} \times \hat{z}|$} \\
\textbf{Type} & \textbf{Code} & \textbf{(GeV)} & \textbf{(rad)} & \textbf{(rad)} & & \\
\hline
"""

pre_table = r"""
\begin{table}[h]
\centering
\caption{Event 7 - Pre-Rotation Particle Properties}
\begin{tabular}{|c|c|c|c|c|c|c|c|c|}
\hline
\textbf{Particle} & \textbf{PDG} & \textbf{Energy} & \textbf{Position} & \textbf{$\theta$} & \textbf{$\phi$} & \textbf{$\vec{p} \cdot \hat{z}$} & \textbf{$|\vec{p} \times \hat{z}|$} \\
\textbf{Type} & \textbf{Code} & \textbf{(GeV)} & \textbf{(m)} & \textbf{(rad)} & \textbf{(rad)} & & \\
\hline
"""

# Add primary neutrino
nu_pos_str = f"({nu_pos_pre[0]:.1f}, {nu_pos_pre[1]:.1f}, {nu_pos_pre[2]:.1f})"
pre_table += f"$\\nu_\\mu$ (primary) & 14 & {nu_energy_pre:.3f} & {nu_pos_str} & {nu_theta_pre:.4f} & -- & 1.000 & 0.000 \\\\\n"
pre_table += "\\hline\n"

# Add child particles
for i in range(len(child_vecs_pre)):
    pdg_name = pdg_names.get(int(pdg_pre[i]), f"{int(pdg_pre[i])}")
    pos_str = f"({pos_pre[i][0]:.1f}, {pos_pre[i][1]:.1f}, {pos_pre[i][2]:.1f})"
    pre_table += f"{pdg_name} & {int(pdg_pre[i])} & {energy_pre[i]:.3f} & {pos_str} & {theta_pre[i]:.4f} & {phi_pre[i]:.4f} & {dots_pre[i]:.3f} & {crosses_pre[i]:.3f} \\\\\n"

pre_table += r"""
\hline
\end{tabular}
\end{table}
"""

print(pre_table)

print("\n" + "="*80)
print("POST-ROTATION TABLE (Event 7)")
print("="*80)

post_table = r"""
\begin{table}[h]
\centering
\caption{Event 7 - Post-Rotation Particle Properties}
\begin{tabular}{|c|c|c|c|c|c|c|c|}
\hline
\textbf{Particle} & \textbf{PDG} & \textbf{Energy} & \textbf{$\theta$} & \textbf{$\phi$} & \textbf{$\vec{p} \cdot \hat{\nu}$} & \textbf{$|\vec{p} \times \hat{\nu}|$} \\
\textbf{Type} & \textbf{Code} & \textbf{(GeV)} & \textbf{(rad)} & \textbf{(rad)} & & \\
\hline
"""

post_table = r"""
\begin{table}[h]
\centering
\caption{Event 7 - Post-Rotation Particle Properties}
\begin{tabular}{|c|c|c|c|c|c|c|c|c|}
\hline
\textbf{Particle} & \textbf{PDG} & \textbf{Energy} & \textbf{Position} & \textbf{$\theta$} & \textbf{$\phi$} & \textbf{$\vec{p} \cdot \hat{\nu}$} & \textbf{$|\vec{p} \times \hat{\nu}|$} \\
\textbf{Type} & \textbf{Code} & \textbf{(GeV)} & \textbf{(m)} & \textbf{(rad)} & \textbf{(rad)} & & \\
\hline
"""

# Add primary neutrino
nu_pos_post_str = f"({nu_pos_post[0]:.1f}, {nu_pos_post[1]:.1f}, {nu_pos_post[2]:.1f})"
post_table += f"$\\nu_\\mu$ (primary) & 14 & {nu_energy_post:.3f} & {nu_pos_post_str} & {nu_theta_post:.4f} & {nu_phi_post:.4f} & 1.000 & 0.000 \\\\\n"
post_table += "\\hline\n"

# Add child particles
for i in range(len(child_vecs_post)):
    pdg_name = pdg_names.get(int(pdg_post[i]), f"{int(pdg_post[i])}")
    pos_post_str = f"({pos_post[i][0]:.1f}, {pos_post[i][1]:.1f}, {pos_post[i][2]:.1f})"
    post_table += f"{pdg_name} & {int(pdg_post[i])} & {energy_post[i]:.3f} & {pos_post_str} & {theta_post[i]:.4f} & {phi_post[i]:.4f} & {dots_post[i]:.3f} & {crosses_post[i]:.3f} \\\\\n"

post_table += r"""
\hline
\end{tabular}
\end{table}
"""

print(post_table)

print("\n" + "="*80)
print("COMPARISON TABLE - Key Observables")
print("="*80)

comparison_table = r"""
\begin{table}[h]
\centering
\caption{Event 7 - Rotation Verification: Preserved Kinematic Relationships}
\begin{tabular}{|c|c|c|c|}
\hline
\textbf{Observable} & \textbf{Pre-Rotation} & \textbf{Post-Rotation} & \textbf{$\Delta$} \\
\hline
"""

comparison_table += f"Primary $\\nu$ direction & $(0, 0, 1)$ & $({nu_vec_post[0]:.3f}, {nu_vec_post[1]:.3f}, {nu_vec_post[2]:.3f})$ & -- \\\\\n"
comparison_table += "\\hline\n"

# Inter-particle angles
for i in range(len(child_vecs_pre)-1):
    for j in range(i+1, len(child_vecs_pre)):
        angle_pre = np.arccos(np.clip(np.dot(child_vecs_pre[i], child_vecs_pre[j]), -1, 1)) * 180/np.pi
        angle_post = np.arccos(np.clip(np.dot(child_vecs_post[i], child_vecs_post[j]), -1, 1)) * 180/np.pi
        pdg1 = pdg_names.get(int(pdg_pre[i]), f"{int(pdg_pre[i])}")
        pdg2 = pdg_names.get(int(pdg_pre[j]), f"{int(pdg_pre[j])}")
        comparison_table += f"$\\angle({pdg1}, {pdg2})$ & ${angle_pre:.2f}°$ & ${angle_post:.2f}°$ & ${abs(angle_pre-angle_post):.4f}°$ \\\\\n"

comparison_table += "\\hline\n"

# Neutrino-relative angles  
for i in range(len(child_vecs_pre)):
    angle_pre = np.arccos(np.clip(dots_pre[i], -1, 1)) * 180/np.pi
    angle_post = np.arccos(np.clip(dots_post[i], -1, 1)) * 180/np.pi
    pdg_name = pdg_names.get(int(pdg_pre[i]), f"{int(pdg_pre[i])}")
    comparison_table += f"$\\angle({pdg_name}, \\nu)$ & ${angle_pre:.2f}°$ & ${angle_post:.2f}°$ & ${abs(angle_pre-angle_post):.4f}°$ \\\\\n"

comparison_table += r"""
\hline
\end{tabular}
\end{table}
"""

print(comparison_table)

print("\n" + "="*60)
print("LaTeX USAGE NOTES:")
print("="*60)
print("1. Add \\usepackage{amsmath} to your LaTeX preamble")
print("2. These tables show the kinematics are perfectly preserved")
print("3. The dot products (ν̂·p̂) show alignment with neutrino direction")
print("4. Position shows the interaction vertex after cylinder injection")
print("5. All relative angles have Δ ≈ 0, confirming rigid rotation!")
print("6. Positions are the same for all particles in an event (shared vertex)")
print("7. Only directions change - energies and vertex positions stay constant!")

PRE-ROTATION TABLE (Event 7)

\begin{table}[h]
\centering
\caption{Event 7 - Pre-Rotation Particle Properties}
\begin{tabular}{|c|c|c|c|c|c|c|c|c|}
\hline
\textbf{Particle} & \textbf{PDG} & \textbf{Energy} & \textbf{Position} & \textbf{$\theta$} & \textbf{$\phi$} & \textbf{$\vec{p} \cdot \hat{z}$} & \textbf{$|\vec{p} \times \hat{z}|$} \\
\textbf{Type} & \textbf{Code} & \textbf{(GeV)} & \textbf{(m)} & \textbf{(rad)} & \textbf{(rad)} & & \\
\hline
$\nu_\mu$ (primary) & 14 & 32.736 & (0.0, 0.0, 0.0) & 0.0000 & -- & 1.000 & 0.000 \\
\hline
$\mu^-$ & 13 & 17.587 & (0.0, 0.0, 0.0) & 0.1305 & 0.2195 & 0.991 & 0.130 \\
$\pi^+$ & 211 & 11.361 & (0.0, 0.0, 0.0) & 0.1411 & 2.8935 & 0.990 & 0.141 \\
$\pi^-$ & -211 & 0.616 & (0.0, 0.0, 0.0) & 0.6123 & 2.6083 & 0.818 & 0.575 \\
$p$ & 2212 & 1.704 & (0.0, 0.0, 0.0) & 0.0327 & 1.7922 & 0.999 & 0.033 \\
$\pi^+$ & 211 & 1.939 & (0.0, 0.0, 0.0) & 0.1498 & 2.6275 & 0.989 & 0.149 \\
111 & 111 & 0.468 & (0.0, 0.0, 0.0) & 0.1056 & 1.6606 & 0.994 & 0.105 \\



In [9]:
# Assuming you have the data loaded as before
event_idx = 0

# Extract data for event 7
nu_theta_pre = primary_set_pre_injection.iloc[event_idx]['theta']
nu_phi_pre = primary_set_pre_injection.iloc[event_idx]['phi']
nu_energy_pre = primary_set_pre_injection.iloc[event_idx]['e']
nu_pos_pre = primary_set_pre_injection.iloc[event_idx]['position']



In [8]:
primary_set_pre_injection.iloc[0]['custom_info']

'nu:14;tgt:1000080160;N:2112;q:1(v);proc:Weak[CC],DIS;'

In [49]:
pdg_pre

array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])

primary         [False, False, False, False, False, False, Fal...
e               [0.00618, 9.63373038253645, 0.5708686471054392...
pdg_code             [22, 13, 22, 22, 211, 3122, 321, 2112, -211]
interaction                  [CC, CC, CC, CC, CC, CC, CC, CC, CC]
theta           [2.169488414463627, 1.3045772954289971, 2.2137...
phi             [2.5242482505162487, -0.04537318952939684, 0.0...
bjorken_x       [-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....
bjorken_y       [-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....
pos_x           [-250.18382169627446, -250.18382169627446, -25...
pos_y           [208.65590118686183, 208.65590118686183, 208.6...
pos_z           [196.710491615592, 196.710491615592, 196.71049...
position        [[-250.18382169627446, 208.65590118686183, 196...
column_depth    [-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....
custom_info     [child, child, child, child, child, child, chi...
Name: 0, dtype: object

primary                                                      True
e                                                       20.945311
pdg_code                                                       14
interaction                                                    CC
theta                                                    1.291457
phi                                                      6.191581
bjorken_x                                                      -1
bjorken_y                                                      -1
pos_x                                                 -250.183822
pos_y                                                  208.655901
pos_z                                                  196.710492
position        (-250.18382169627446, 208.65590118686183, 196....
column_depth                                                   -1
custom_info     nu:14;tgt:1000080160;N:2112;q:1(v);proc:Weak[C...
Name: 0, dtype: object

In [40]:
primary_set_pre_injection.iloc[7]

primary                                                      True
e                                                       31.404678
pdg_code                                                       14
interaction                                                    CC
theta                                                         0.0
phi                                                           NaN
bjorken_x                                                      -1
bjorken_y                                                      -1
pos_x                                                         0.0
pos_y                                                         0.0
pos_z                                                         0.0
position                                          [0.0, 0.0, 0.0]
column_depth                                                   -1
custom_info     nu:14;tgt:1000010010;N:2212;proc:Weak[CC],RES;...
Name: 7, dtype: object

In [39]:
prometheus_set_pre_injection.iloc[7]

primary                                     [False, False, False]
e               [31.117647204712135, 1.0256771859269784, 0.199...
pdg_code                                          [13, 2212, 211]
interaction                                          [CC, CC, CC]
theta           [0.007491207223070428, 0.7152524102626427, 1.9...
phi             [2.3324912191531064, 0.3816609421965292, 1.781...
bjorken_x                                      [-1.0, -1.0, -1.0]
bjorken_y                                      [-1.0, -1.0, -1.0]
pos_x           [421.8273127589046, 421.8273127589046, 421.827...
pos_y           [-181.45359865842642, -181.45359865842642, -18...
pos_z           [-177.08513203279455, -177.08513203279455, -17...
position        [[421.8273127589046, -181.45359865842642, -177...
column_depth                                   [-1.0, -1.0, -1.0]
custom_info                                 [child, child, child]
Name: 7, dtype: object

In [38]:
prometheus_set.iloc[7]

primary                                     [False, False, False]
e               [31.117647204712135, 1.0256771859269784, 0.199...
pdg_code                                          [13, 2212, 211]
interaction                                          [CC, CC, CC]
theta           [2.124997323094431, 2.843055702355283, 1.48099...
phi             [0.3853336665029423, 0.3871871743611579, 2.394...
bjorken_x                                      [-1.0, -1.0, -1.0]
bjorken_y                                      [-1.0, -1.0, -1.0]
pos_x           [421.8273127589046, 421.8273127589046, 421.827...
pos_y           [-181.45359865842642, -181.45359865842642, -18...
pos_z           [-177.08513203279455, -177.08513203279455, -17...
position        [[421.8273127589046, -181.45359865842642, -177...
column_depth                                   [-1.0, -1.0, -1.0]
custom_info                                 [child, child, child]
Name: 7, dtype: object

In [31]:
primary_set_pre_injection.iloc[8]

primary                                                      True
e                                                       53.624538
pdg_code                                                       14
interaction                                                    CC
theta                                                         0.0
phi                                                           NaN
bjorken_x                                                      -1
bjorken_y                                                      -1
pos_x                                                         0.0
pos_y                                                         0.0
pos_z                                                         0.0
position                                          [0.0, 0.0, 0.0]
column_depth                                                   -1
custom_info     nu:14;tgt:1000080160;N:2212;q:1(v);proc:Weak[C...
Name: 8, dtype: object

In [53]:
for i in range(len(prometheus_set_pre_injection)):
    print(len(prometheus_set_pre_injection.iloc[i]['e']))

9
18
4
4
12
5
8
3
7
5


In [46]:
prometheus_set_pre_injection.iloc[3]

primary                              [False, False, False, False]
e               [0.00632, 0.15330947054706892, 2.0611488336339...
pdg_code                                      [22, 13, 2212, 211]
interaction                                      [CC, CC, CC, CC]
theta           [3.113677762659773, 1.6520491749936623, 0.1446...
phi             [1.5937204959414146, 2.8005873259375242, 0.979...
bjorken_x                                [-1.0, -1.0, -1.0, -1.0]
bjorken_y                                [-1.0, -1.0, -1.0, -1.0]
pos_x           [-204.70773688800037, -204.70773688800037, -20...
pos_y           [-176.73631430289637, -176.73631430289637, -17...
pos_z           [-70.91195315709973, -70.91195315709973, -70.9...
position        [[-204.70773688800037, -176.73631430289637, -7...
column_depth                             [-1.0, -1.0, -1.0, -1.0]
custom_info                          [child, child, child, child]
Name: 3, dtype: object

In [48]:
prometheus_set.iloc[3]

primary                              [False, False, False, False]
e               [0.00632, 0.15330947054706892, 2.0611488336339...
pdg_code                                      [22, 13, 2212, 211]
interaction                                      [CC, CC, CC, CC]
theta           [1.2150699343880176, 0.33623843065102454, 1.91...
phi             [2.5559742213179697, -1.279660861632222, -0.40...
bjorken_x                                [-1.0, -1.0, -1.0, -1.0]
bjorken_y                                [-1.0, -1.0, -1.0, -1.0]
pos_x           [-204.70773688800037, -204.70773688800037, -20...
pos_y           [-176.73631430289637, -176.73631430289637, -17...
pos_z           [-70.91195315709973, -70.91195315709973, -70.9...
position        [[-204.70773688800037, -176.73631430289637, -7...
column_depth                             [-1.0, -1.0, -1.0, -1.0]
custom_info                          [child, child, child, child]
Name: 3, dtype: object

In [8]:

primary_set_path = f'{OUTPUT_DIR}/genie_events_primary_new_test.parquet'
prometheus_set_path = f'{OUTPUT_DIR}/genie_events_prometheus_new_test.parquet'
out_parquet =  f"{OUTPUT_DIR}/{num_events}_events_simset_{simset}.parquet"

prometheus_set = pd.read_parquet(prometheus_set_path)
primary_set = pd.read_parquet(primary_set_path)
out_parquet = pd.read_parquet(out_parquet)

### the position on the primary set and prometheus set don't have the ~1950m z offset btw

In [8]:
primary_set

,primary,e,pdg_code,interaction,theta,phi,bjorken_x,bjorken_y,pos_x,pos_y,pos_z,position,column_depth,custom_info
0,True,20.945311,14,CC,0.184551,6.139557,-1,-1,72.599180,226.243706,-229.834532,"[72.59917981840435, 226.24370572146523, -229.8...",-1,nu:14;tgt:1000080160;N:2112;q:1(v);proc:Weak[C...
1,True,6.593265,14,CC,1.849128,0.027354,-1,-1,299.036303,152.221651,-327.326402,"[299.03630313318274, 152.221651275712, -327.32...",-1,nu:14;tgt:1000080160;N:2212;q:1(v);proc:Weak[C...
2,True,2.100276,14,NC,0.280672,0.431699,-1,-1,-362.487255,-285.751483,69.775968,"[-362.48725485708013, -285.7514829630307, 69.7...",-1,nu:14;tgt:1000080160;N:2112;q:1(v);proc:Weak[N...
3,True,1.784836,14,CC,0.997281,2.402284,-1,-1,-354.572521,68.021462,-378.513661,"[-354.5725208289885, 68.02146247858087, -378.5...",-1,"nu:14;tgt:1000080160;N:2212;proc:Weak[CC],RES;..."
4,True,76.670259,14,CC,1.483880,0.263696,-1,-1,-63.833110,-381.960740,-425.126891,"[-63.83310951748334, -381.9607404728206, -425....",-1,nu:14;tgt:1000080160;N:2112;q:1(v);proc:Weak[C...
5,True,8.242408,14,NC,2.517815,5.005414,-1,-1,70.631362,176.460485,365.822954,"[70.63136232750487, 176.46048513988993, 365.82...",-1,nu:14;tgt:1000080160;N:2212;q:2(v);proc:Weak[N...
6,True,5.205773,14,CC,1.381477,4.464592,-1,-1,99.668669,180.293026,-322.412156,"[99.66866903885621, 180.29302615569037, -322.4...",-1,nu:14;tgt:1000080160;N:2212;q:1(v);proc:Weak[C...
7,True,31.404678,14,CC,0.421972,0.450229,-1,-1,205.660168,37.437626,38.678779,"[205.66016762422652, 37.437625666864186, 38.67...",-1,"nu:14;tgt:1000010010;N:2212;proc:Weak[CC],RES;..."
8,True,53.624538,14,CC,2.105694,5.947884,-1,-1,-204.661731,215.004565,312.908555,"[-204.66173104338043, 215.00456526331513, 312....",-1,nu:14;tgt:1000080160;N:2212;q:1(v);proc:Weak[C...
9,True,6.006226,14,CC,1.918054,4.801989,-1,-1,-93.877529,331.880016,155.356507,"[-93.87752861072676, 331.88001612458294, 155.3...",-1,nu:14;tgt:1000080160;N:2212;q:1(s);proc:Weak[C...


In [8]:
primary_set

,primary,e,pdg_code,interaction,theta,phi,bjorken_x,bjorken_y,pos_x,pos_y,pos_z,position,column_depth,custom_info
0,True,4.120269,14,NC,1.798786,3.952155,-1,-1,-55.276542,411.979621,93.401069,"[-55.27654191196277, 411.97962084633053, 93.40...",-1,"nu:14;tgt:1000080160;N:2212;proc:Weak[NC],QES;"
1,True,33.232259,14,NC,2.275174,2.543965,-1,-1,158.436050,-454.016293,-470.303702,"[158.43605044672566, -454.0162933842299, -470....",-1,nu:14;tgt:1000010010;N:2212;q:2(v);proc:Weak[N...
2,True,26.600404,14,CC,0.957754,1.541885,-1,-1,303.299883,189.332464,67.266637,"[303.2998829928405, 189.3324636443426, 67.2666...",-1,nu:14;tgt:1000080160;N:2112;q:1(v);proc:Weak[C...
3,True,31.335799,14,CC,2.341539,2.257375,-1,-1,-121.592327,191.978043,266.947901,"[-121.59232735308856, 191.9780433161717, 266.9...",-1,nu:14;tgt:1000080160;N:2112;q:1(v);proc:Weak[C...
4,True,1.904728,14,CC,1.220930,0.399815,-1,-1,-30.530031,-486.242938,28.529813,"[-30.53003127175284, -486.24293756252, 28.5298...",-1,"nu:14;tgt:1000080160;N:2212;proc:Weak[CC],RES;..."
5,True,2.360738,14,CC,1.538925,1.322801,-1,-1,-58.628804,-244.307897,-283.249035,"[-58.62880447740481, -244.3078974893482, -283....",-1,"nu:14;tgt:1000080160;N:2112;proc:Weak[CC],RES;..."
6,True,3.968209,14,CC,1.851010,5.970824,-1,-1,289.657283,-44.460051,355.799262,"[289.6572833170811, -44.46005060419492, 355.79...",-1,"nu:14;tgt:1000080160;N:2112;proc:Weak[CC],QES;"
7,True,36.513016,14,NC,2.013351,1.946821,-1,-1,-9.999690,-298.128489,-306.390413,"[-9.999690330843839, -298.1284890956606, -306....",-1,nu:14;tgt:1000080160;N:2112;q:1(v);proc:Weak[N...
8,True,19.521958,14,CC,1.289738,0.920280,-1,-1,-175.472267,-203.294560,-171.663509,"[-175.4722674603497, -203.29456005424524, -171...",-1,nu:14;tgt:1000080160;N:2112;q:1(v);proc:Weak[C...
9,True,2.421808,14,CC,0.880528,5.609453,-1,-1,219.738122,262.889129,-486.620568,"[219.7381220060781, 262.8891286505983, -486.62...",-1,nu:14;tgt:1000080160;N:2212;q:1(v);proc:Weak[C...


In [9]:
prometheus_set

,primary,e,pdg_code,interaction,theta,phi,bjorken_x,bjorken_y,pos_x,pos_y,pos_z,position,column_depth,custom_info
0,"[False, False, False, False]","[0.00444, 3.9605257185188063, 1.05783024823171...","[22, 14, 2212, 2000000101]","[NC, NC, NC, NC]","[1.6575593216954645, 2.0394855167357426, 3.043...","[2.755463690177905, 2.958103217208648, 0.40732...","[-1.0, -1.0, -1.0, -1.0]","[-1.0, -1.0, -1.0, -1.0]","[-368.72103578864704, -368.72103578864704, -36...","[88.45684952547404, 88.45684952547404, 88.4568...","[421.2455614643902, 421.2455614643902, 421.245...","[[-368.72103578864704, 88.45684952547404, 421....","[-1.0, -1.0, -1.0, -1.0]","[child, child, child, child]"
1,"[False, False, False, False, False, False]","[25.744678284252732, 1.5912865794041986, 5.187...","[14, 111, 2112, 111, 211, 111]","[NC, NC, NC, NC, NC, NC]","[1.4384030391588014, 1.404877901441277, 1.3568...","[-0.0963621052585267, -0.07814883702408411, -0...","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0]","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0]","[212.46167245699337, 212.46167245699337, 212.4...","[434.85496449183177, 434.85496449183177, 434.8...","[149.9252355369464, 149.9252355369464, 149.925...","[[212.46167245699337, 434.85496449183177, 149....","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0]","[child, child, child, child, child, child]"
2,"[False, False, False, False, False, False, Fal...","[14.705120183999083, 0.5559631923089461, 2.614...","[13, 111, 2112, 211, 111, 111, 2212, -211]","[CC, CC, CC, CC, CC, CC, CC, CC]","[2.0757049374486694, 2.0183306199295092, 2.071...","[-0.748282774086667, -0.582203690913106, -0.74...","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0]","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0]","[89.85121132316773, 89.85121132316773, 89.8512...","[382.5405518872392, 382.5405518872392, 382.540...","[-360.93719230603983, -360.93719230603983, -36...","[[89.85121132316773, 382.5405518872392, -360.9...","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0]","[child, child, child, child, child, child, chi..."
3,"[False, False, False, False, False, False, False]","[19.904673998313992, 0.6045963532376314, 3.685...","[13, 211, 211, -211, 111, 2112, 111]","[CC, CC, CC, CC, CC, CC, CC]","[0.7389864357426194, 0.793927515664132, 0.6975...","[-1.3681488189786675, -1.8664419360351454, -1....","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0]","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0]","[-71.86357315244918, -71.86357315244918, -71.8...","[440.2595330400279, 440.2595330400279, 440.259...","[-23.267042885691012, -23.267042885691012, -23...","[[-71.86357315244918, 440.2595330400279, -23.2...","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0]","[child, child, child, child, child, child, child]"
4,"[False, False, False, False]","[0.00385, 0.4985908788035815, 1.86633738306353...","[22, 13, 2212, 211]","[CC, CC, CC, CC]","[1.0182224070010635, 0.47839169703966467, 2.06...","[-2.8272527881868497, -1.4927317434681866, -2....","[-1.0, -1.0, -1.0, -1.0]","[-1.0, -1.0, -1.0, -1.0]","[106.17380790173436, 106.17380790173436, 106.1...","[218.6844670480369, 218.6844670480369, 218.684...","[474.88643123575093, 474.88643123575093, 474.8...","[[106.17380790173436, 218.6844670480369, 474.8...","[-1.0, -1.0, -1.0, -1.0]","[child, child, child, child]"
5,"[False, False, False, False, False, False, False]","[0.00618, 1.0975641954166295, 1.12105649722162...","[22, 13, 2112, 2212, 2212, 211, -211]","[CC, CC, CC, CC, CC, CC, CC]","[1.5093383373311102, 0.8190975074433162, 1.643...","[1.996303249691743, 0.23405544680603085, 0.570...","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0]","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0]","[158.3586568004207, 158.3586568004207, 158.358...","[204.66668388906854, 204.66668388906854, 204.6...","[-397.9373333381465, -397.9373333381465, -397....","[[158.3586568004207, 204.66668388906854, -397....","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0]","[child, child, child, child, child, child, child]"
6,"[False, False, False, False, False, False, Fal...","[2.9087030454378104, 0.96783604951397

In [9]:
prometheus_set

,primary,e,pdg_code,interaction,theta,phi,bjorken_x,bjorken_y,pos_x,pos_y,pos_z,position,column_depth,custom_info
0,"[False, False, False, False]","[0.00444, 3.9605257185188063, 1.05783024823171...","[22, 14, 2212, 2000000101]","[NC, NC, NC, NC]","[1.33954512405679, 1.7270628274994135, 2.15283...","[-2.081085080647074, -2.2642693549578436, 2.80...","[-1.0, -1.0, -1.0, -1.0]","[-1.0, -1.0, -1.0, -1.0]","[-55.27654191196277, -55.27654191196277, -55.2...","[411.97962084633053, 411.97962084633053, 411.9...","[93.401069358727, 93.401069358727, 93.40106935...","[[-55.27654191196277, 411.97962084633053, 93.4...","[-1.0, -1.0, -1.0, -1.0]","[child, child, child, child]"
1,"[False, False, False, False, False, False]","[25.744678284252732, 1.5912865794041986, 5.187...","[14, 111, 2112, 111, 211, 111]","[NC, NC, NC, NC, NC, NC]","[2.278082089161442, 2.31610097193389, 2.380406...","[2.5019387453639834, 2.499674526888428, 2.4665...","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0]","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0]","[158.43605044672566, 158.43605044672566, 158.4...","[-454.0162933842299, -454.0162933842299, -454....","[-470.3037021882566, -470.3037021882566, -470....","[[158.43605044672566, -454.0162933842299, -470...","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0]","[child, child, child, child, child, child]"
2,"[False, False, False, False, False, False, Fal...","[14.705120183999083, 0.5559631923089461, 2.614...","[13, 111, 2112, 211, 111, 111, 2212, -211]","[CC, CC, CC, CC, CC, CC, CC, CC]","[0.9607146863364877, 1.1081232183010419, 0.966...","[1.5421143737263365, 1.4748814407446178, 1.542...","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0]","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0]","[303.2998829928405, 303.2998829928405, 303.299...","[189.3324636443426, 189.3324636443426, 189.332...","[67.2666369257222, 67.2666369257222, 67.266636...","[[303.2998829928405, 189.3324636443426, 67.266...","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0]","[child, child, child, child, child, child, chi..."
3,"[False, False, False, False, False, False, False]","[19.904673998313992, 0.6045963532376314, 3.685...","[13, 211, 211, -211, 111, 2112, 111]","[CC, CC, CC, CC, CC, CC, CC]","[2.3123686217900015, 2.4808108572065737, 2.484...","[2.100657013058277, 2.5542346310900146, 2.3164...","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0]","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0]","[-121.59232735308856, -121.59232735308856, -12...","[191.9780433161717, 191.9780433161717, 191.978...","[266.94790069775, 266.94790069775, 266.9479006...","[[-121.59232735308856, 191.9780433161717, 266....","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0]","[child, child, child, child, child, child, child]"
4,"[False, False, False, False]","[0.00385, 0.4985908788035815, 1.86633738306353...","[22, 13, 2212, 211]","[CC, CC, CC, CC]","[2.0019486343945605, 2.673971728274342, 0.9588...","[0.6210345917898809, -0.48204963689094044, 0.5...","[-1.0, -1.0, -1.0, -1.0]","[-1.0, -1.0, -1.0, -1.0]","[-30.53003127175284, -30.53003127175284, -30.5...","[-486.24293756252, -486.24293756252, -486.2429...","[28.529812841704597, 28.529812841704597, 28.52...","[[-30.53003127175284, -486.24293756252, 28.529...","[-1.0, -1.0, -1.0, -1.0]","[child, child, child, child]"
5,"[False, False, False, False, False, False, False]","[0.00618, 1.0975641954166295, 1.12105649722162...","[22, 13, 2112, 2212, 2212, 211, -211]","[CC, CC, CC, CC, CC, CC, CC]","[2.506505778316478, 1.7576894560941592, 2.3190...","[-2.9407819062461362, 1.3352164743506405, 0.55...","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0]","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0]","[-58.62880447740481, -58.62880447740481, -58.6...","[-244.3078974893482, -244.3078974893482, -244....","[-283.24903520351353, -283.24903520351353, -28...","[[-58.62880447740481, -244.3078974893482, -283...","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0]","[child, child, child, child, child, child, child]"
6,"[False, False, False, False, False, False, Fal...","[2.9087030454378104, 0.96783604951397

### and the normal mc_truth and photons fields from the parquet:


In [10]:
out_parquet

,mc_truth,photons
0,"{'interaction': 2, 'initial_state_energy': 4.1...","{'sensor_pos_x': [], 'sensor_pos_y': [], 'sens..."
1,"{'interaction': 2, 'initial_state_energy': 33....","{'sensor_pos_x': [224.58], 'sensor_pos_y': [43..."
2,"{'interaction': 1, 'initial_state_energy': 26....","{'sensor_pos_x': [174.47, 101.04], 'sensor_pos..."
3,"{'interaction': 1, 'initial_state_energy': 31....","{'sensor_pos_x': [-21.97], 'sensor_pos_y': [39..."
4,"{'interaction': 1, 'initial_state_energy': 1.9...","{'sensor_pos_x': [54.26], 'sensor_pos_y': [292..."
5,"{'interaction': 1, 'initial_state_energy': 2.3...","{'sensor_pos_x': [195.03], 'sensor_pos_y': [12..."
6,"{'interaction': 1, 'initial_state_energy': 3.9...","{'sensor_pos_x': [114.39, 158.94], 'sensor_pos..."
7,"{'interaction': 2, 'initial_state_energy': 36....","{'sensor_pos_x': [], 'sensor_pos_y': [], 'sens..."
8,"{'interaction': 1, 'initial_state_energy': 19....","{'sensor_pos_x': [-224.09, -145.45, -224.09, -..."
9,"{'interaction': 1, 'initial_state_energy': 2.4...","{'sensor_pos_x': [210.47, 46.29], 'sensor_pos_..."


In [11]:
out_parquet['mc_truth'][2]

{'interaction': 1,
 'initial_state_energy': 26.600404392730383,
 'initial_state_type': 14,
 'initial_state_zenith': 2.077803479928837,
 'initial_state_azimuth': -0.7506804902632054,
 'initial_state_x': 95.72204078053198,
 'initial_state_y': 380.02194336010433,
 'initial_state_z': -2332.912957809923,
 'final_state_energy': array([1.47051202e+01, 1.42527101e-02, 4.12656699e-02, 5.01399945e-02,
        5.55963192e-01, 2.61423463e+00, 7.62118454e-01, 7.30616473e+00,
        3.44948570e-01, 9.55530679e-01, 1.19694698e+00]),
 'final_state_type': array([  13.,   11.,   14.,  -12.,  111., 2112.,  211.,  111.,  111.,
        2212., -211.]),
 'final_state_zenith': array([2.07570494, 1.38551193, 2.13226019, 1.17465952, 2.01833062,
        2.07107462, 1.81318441, 2.06932305, 2.12027141, 2.02442431,
        1.94066615]),
 'final_state_azimuth': array([-0.74828277,  1.47899129,  2.21695426, -1.12969056, -0.58220369,
        -0.74339616, -0.92648401, -0.73127079,  1.47319115, -0.697488  ,
        -0.

In [12]:
out_parquet['photons'][2]

{'sensor_pos_x': array([174.47, 101.04]),
 'sensor_pos_y': array([315.54, 412.79]),
 'sensor_pos_z': array([-2422.53, -2345.75]),
 'string_id': array([65, 72]),
 'sensor_id': array([59, 54]),
 't': array([765.78894 , 286.842651]),
 'id_idx': array([1, 1])}

#### so I guess you don't need to load the prometheus and primary set if you have the mc_truth, but maybe something to keep in mind for one weight or anything needed to reweight to flux